In [225]:
from config import *

# 0. Data

In [226]:
df = pd.read_csv('data/italian_stock_data_2004_2024.csv', index_col=0, header=[0,1])
df = df.stack(level='Ticker').reset_index()
df.fillna(method='ffill', inplace=True) 
df.Date = pd.to_datetime(df.Date)
df.rename(columns={col:col.lower() for col in df.columns}, inplace=True)
df.rename(columns={'adj close':'adj_close'}, inplace=True)
df.ticker = df.ticker.str.replace('.MI','').str.replace('1','')

/var/folders/cd/gw01nxmn4d513frhd6cyw6080000gp/T/ipykernel_46318/1465876591.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df.ticker = df.ticker.str.replace('.MI','').str.replace('1','')


Feature engineering

In [227]:
# Basic Features
df['daily_return'] = df.groupby('ticker')['adj_close'].pct_change()
df['price_range'] = df['high'] - df['low']

# Moving Averages
df['sma_20'] = df.groupby('ticker')['adj_close'].rolling(window=20).mean().reset_index(0, drop=True)
df['sma_50'] = df.groupby('ticker')['adj_close'].rolling(window=50).mean().reset_index(0, drop=True)
df['ema_20'] = df.groupby('ticker', group_keys=False)['adj_close'].apply(lambda x: x.ewm(span=20, adjust=False).mean())

# Volatility Features
df['volatility_20'] = df.groupby('ticker')['adj_close'].rolling(window=20).std().reset_index(0, drop=True)
df['bollinger_upper'] = df['sma_20'] + 2 * df['volatility_20']
df['bollinger_lower'] = df['sma_20'] - 2 * df['volatility_20']

# Momentum Indicators
delta = df.groupby('ticker')['adj_close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
df['rsi'] = 100 - (100 / (1 + gain / loss))
df['macd'] = df.groupby('ticker', group_keys=False)['adj_close'].apply(
    lambda x: x.ewm(span=12, adjust=False).mean() - x.ewm(span=26, adjust=False).mean()
).reset_index(level=0, drop=True)
df['signal'] = df['macd'].ewm(span=9, adjust=False).mean()

# Volume Features
df['volume_sma_20'] = df.groupby('ticker')['volume'].rolling(window=20).mean().reset_index(0, drop=True)
#df['volume_spike'] = df['volume'] > 2 * df['volume_sma_20']

# Custom Features for GP
df['open_close_ratio'] = df['open'] / df['close']
df['high_low_ratio'] = df['high'] / df['low']
#df['day_of_week'] = pd.to_datetime(df['date']).dt.dayofweek
df['cumulative_return'] = (1 + df['daily_return']).groupby(df['ticker']).cumprod()

In [228]:
df.sort_values(['ticker','date'], inplace=True)
df.set_index('date', inplace=True)
df['year'] = df.index.year

In [229]:
features = ['adj_close', 'close', 'high', 'low', 'open', 'volume',
       'daily_return', 'price_range', 'sma_20', 'sma_50', 'ema_20',
       'volatility_20', 'bollinger_upper', 'bollinger_lower', 'rsi', 'macd',
       'signal', 'volume_sma_20', 'open_close_ratio',
       'high_low_ratio', 'cumulative_return']

In [230]:
def standardize_features(group_df):
    for feature in features:
        group_df[feature] = (
            group_df[feature] - group_df[feature].mean()
        ) / group_df[feature].std()
    return group_df

In [231]:
df = df.groupby('ticker', group_keys=False).apply(standardize_features)
df = df.dropna()
df = df[~df.ticker.isin(['LDO','MONC','PIRC','PRY','PST'])] # these stocks appear later in the data, we want to start running the algorithm from 2004
df

Price,ticker,adj_close,close,high,low,open,volume,daily_return,price_range,sma_20,...,bollinger_upper,bollinger_lower,rsi,macd,signal,volume_sma_20,open_close_ratio,high_low_ratio,cumulative_return,year
date,,,,,,,,,,,,,,,,,,,,,
2004-03-10 00:00:00+00:00,A2A,-0.733183,0.271483,0.286225,0.285612,0.317797,-0.923188,-0.715508,0.136278,-0.750584,...,-0.789671,-0.701981,0.188346,0.288471,0.352079,-1.523189,0.883551,-0.146328,-0.733323,2004
2004-03-11 00:00:00+00:00,A2A,-0.771611,0.204841,0.238975,0.208812,0.270181,-0.793142,-1.251627,0.897880,-0.750861,...,-0.789558,-0.702676,-0.668306,0.192870,0.392050,-1.496860,1.285478,0.558908,-0.771749,2004
2004-03-12 00:00:00+00:00,A2A,-0.782589,0.185800,0.182274,0.132012,0.174950,-0.889592,-0.380966,1.405619,-0.750861,...,-0.789558,-0.702676,-1.413255,0.097713,0.383032,-1.473887,-0.238113,1.074767,-0.782728,2004
2004-03-15 00:00:00+00:00,A2A,-0.804548,0.147719,0.172824,0.160812,0.174950,-0.985167,-0.745276,0.390142,-0.752246,...,-0.787226,-0.708055,-1.375669,-0.012989,0.351354,-1.457927,0.535766,0.145312,-0.804686,2004
2004-03-16 00:00:00+00:00,A2A,-0.816625,0.126774,0.133134,0.135852,0.119716,-1.017320,-0.424880,-0.016050,-0.754240,...,-0.783718,-0.715962,-1.413255,-0.119331,0.160064,-1.446432,-0.163896,-0.202684,-0.816763,2004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-21 00:00:00+00:00,UCG,-0.375302,-0.525852,-0.531877,-0.521731,-0.528314,-0.840307,-0.015819,-0.701834,-0.360223,...,-0.388511,-0.327368,0.441928,0.037390,0.474876,-0.204950,-0.212031,-0.808740,-0.375121,2023
2023-12-22 00:00:00+00:00,UCG,-0.372973,-0.524343,-0.532532,-0.520210,-0.525863,-0.912924,0.095814,-0.783210,-0.361243,...,-0.389313,-0.328629,0.606483,0.031644,0.488487,-0.222503,-0.129531,-1.006864,-0.372791,2023
2023-12-27 00:00:00+00:00,UCG,-0.372099,-0.523777,-0.529819,-0.519164,-0.524261,-0.958765,0.030017,-0.719524,-0.361745,...,-0.389484,-0.329499,0.974695,0.029707,0.500576,-0.249726,-0.039342,-0.854408,-0.371918,2023


In [232]:
ticker_symbols = df.ticker.unique()

# 1. Portfolio selection using GP

## Baseline: Markowitz

In [233]:
df_mark =  df[df.index<'2019-01-01'].pivot(columns='ticker', values='adj_close')
num_stocks = len(df_mark.columns)
avg_returns = np.array([np.mean(df_mark[col]) for col in df_mark.columns]).reshape(num_stocks,1)
cov_matrix = np.matrix(df_mark.cov())

In [234]:
optimal_weights_markowitz, optimal_fitness_markowitz = markowitz_solution(num_stocks,
                                                                          avg_returns,
                                                                          cov_matrix,
                                                                          risk_aversion=0.5,
                                                                          short_selling=True)

## Genetic programming

In [235]:
# Parameters for genetic programming
POPULATION_SIZE = 10
GENERATIONS = 20
TOURNAMENT_SIZE = 3
MUTATION_RATE = 0.2

In [236]:
def generate_random_strategy():
    """Generate a random strategy as a simple decision rule."""
    return {
        "threshold": np.random.normal(loc=0, scale=1), # all features have been standardized
        "feature": random.choice(features),
        "type": random.choice(["momentum", "contrarian"]),
    }

In [237]:
def create_portfolio(strategy, df_reference):
    buy_intensities = {}
    for ticker in ticker_symbols:
        if strategy["type"] == "momentum":
            buy_intensities[ticker] = df_reference[df_reference.ticker==ticker][strategy["feature"]].iloc[0] - strategy["threshold"]
        elif strategy["type"] == "contrarian":
            buy_intensities[ticker] = strategy["threshold"] - df_reference[df_reference.ticker==ticker][strategy["feature"]].iloc[0]
    portfolio_weights = np.array(list(buy_intensities.values())) / sum(buy_intensities.values()) # works only if sum is nonzero - shouldn't happen in general
    return portfolio_weights

In [238]:
def evaluate_portfolio(portfolio_weights, df_reference):
    portfolio_value = np.dot(portfolio_weights,df_reference['adj_close'])
    return portfolio_value

### GP operators

In [239]:
def mutate_strategy(strategy):
    """Randomly modify the strategy to introduce variation."""
    if random.random() < MUTATION_RATE:
        strategy["threshold"] += random.uniform(-0.15, 0.15)
    if random.random() < MUTATION_RATE:
        strategy["feature"] = random.choice(features)
    if random.random() < MUTATION_RATE:
        strategy["type"] = "momentum" if strategy["type"] == "contrarian" else "contrarian"
    return strategy

In [240]:
def crossover_strategy(parent1, parent2):
    """Combine two parent strategies to create an offspring."""
    return {
        "threshold": random.choice([parent1["threshold"], parent2["threshold"]]),
        "feature": random.choice([parent1["feature"], parent2["feature"]]),
        "type": random.choice([parent1["type"], parent2["type"]]),
    }

In [241]:
def select_parent(population, fitnesses):
    """Select a parent using tournament selection."""
    tournament = random.sample(list(zip(population, fitnesses)), TOURNAMENT_SIZE)
    return max(tournament, key=lambda x: x[1])[0]

### GP loop

In [242]:
population = [generate_random_strategy() for _ in range(POPULATION_SIZE)]

for year in range(2004,2019):

    # get prices - the prices of this year
    df_reference = df[df.year==year].groupby('ticker').mean()[features].reset_index()
    portfolios = [create_portfolio(strategy, df_reference) for strategy in population]

    for generation in range(GENERATIONS):
        # Evaluate fitness (portfolio value)
        fitnesses = [evaluate_portfolio(portfolio, df_reference) for portfolio in portfolios]

        # Create new population
        new_population = []
        
        for _ in range(POPULATION_SIZE):
            
            # Select parents
            parent1 = select_parent(population, fitnesses)
            parent2 = select_parent(population, fitnesses)

            # Create offspring
            offspring = crossover_strategy(parent1, parent2)
            offspring = mutate_strategy(offspring)

            new_population.append(offspring)

        population = new_population

# 2. Plots

In [243]:
def plot_portfolio_value_evolution(portfolio_history):
    pass